## PROJETO AUTOMAÇÕES DE PROCESSOS CALCULANDO INDICADORES DE LOJAS E ENVIANDO POR EMAIL PARA GERENTES E DIRETORES DA EMPRESA
### FEITO POR GUILHERME MARTINS

In [12]:
import pandas as pd 
import pathlib
import pythoncom
import email.message
import mimetypes
import smtplib
from email.message import EmailMessage



pythoncom.CoInitialize()

emails = pd.read_excel(r'Bases de Dados\Emails.xlsx')
lojas = pd.read_csv(r'Bases de Dados\Lojas.csv',encoding='latin1', sep=';')
vendas = pd.read_excel(r'Bases de Dados\Vendas.xlsx')


###  Criando uma tabela para cada loja e definindo o dia indicador

In [13]:
# INCLUIR NOME DA LOJA NA TABELA VENDAS
vendas = vendas.merge(lojas, on='ID Loja')

# CRIAR UMA TABELA PARA CADA LOJA
dic_loja = {}
for loja in lojas['Loja']:
    dic_loja[loja] = vendas.loc[vendas['Loja']==loja, :]

# DEFININDO O DIA DO INDICADOR
dia_indicador = vendas['Data'].max()


### Salvando a planilha na pasta de backup

In [5]:
# IDENTIFICAR SE A PASTA JÁ EXISTE
caminho_backup = pathlib.Path(r'Backup Arquivos Lojas')

arquivos_pasta_backup = caminho_backup.iterdir()
lista_nomes_backp = [arquivo.name for arquivo in arquivos_pasta_backup]

# CRIANDO UMA PASTA PARA CADA LOJA NA PASTA DE BACKUPS
for loja in dic_loja:
    if loja not in lista_nomes_backp:
        nova_pasta = caminho_backup / loja 
        nova_pasta.mkdir()
    
    #CRIANDO OS ARQUIVOS DE CADA LOJA EM SUA RESPECTIVA PASTA
    nome_arquivo = '{}_{}_{}.xlsx'.format(dia_indicador.month, dia_indicador.day, loja)
    local_arquivo = caminho_backup / loja / nome_arquivo
    dic_loja[loja].to_excel(local_arquivo)

In [6]:
# DEFINIÇÃO DE METAS
meta_faturamento_dia = 1000
meta_faturamento_ano = 1650000
meta_qtdeprodutos_dia = 4
meta_qtdeprodutos_ano = 120
meta_ticketmedia_dia = 500
meta_ticketmedia_ano = 500

### Calculando o indicador de 1 loja

In [ ]:
for loja in dic_loja:
    vendas_loja = dic_loja[loja]
    vendas_loja_dia = vendas_loja.loc[vendas_loja['Data']== dia_indicador, :]

    #faturamento
    faturamento_ano = vendas_loja['Valor Final'].sum()
    faturamento_dia = vendas_loja_dia['Valor Final'].sum()

    #diversidade de produtos
    qtde_produtos_ano = len(vendas_loja['Produto'].unique())
    qtde_produtos_dia = len(vendas_loja_dia['Produto'].unique())

    #ticket médio
    valor_venda = vendas_loja.groupby('Código Venda').sum(numeric_only=True)
    ticket_medio_ano = valor_venda['Valor Final'].mean()
    valor_venda_dia = vendas_loja_dia.groupby('Código Venda').sum(numeric_only=True)
    ticket_medio_dia = valor_venda_dia['Valor Final'].mean()

    # ENVIAR EMAILS

    if faturamento_dia >= meta_faturamento_dia:
        cor_fat_dia = "green"
    else:
        cor_fat_dia = "red"
    if faturamento_ano >= meta_faturamento_ano:
        cor_fat_ano = "green"
    else:
        cor_fat_ano = "red"
    if qtde_produtos_dia >= meta_qtdeprodutos_dia:
        cor_qtde_dia = "green"
    else:
        cor_qtde_dia = "red"
    if qtde_produtos_ano >= meta_qtdeprodutos_ano:
        cor_qtde_ano = "green"
    else:
        cor_qtde_ano = "red"
    if ticket_medio_dia >= meta_ticketmedia_dia:
        cor_ticket_dia = "green"
    else:
        cor_ticket_dia = "red"
    if ticket_medio_ano >= meta_ticketmedia_ano:
        cor_ticket_ano = "green"
    else:
        cor_ticket_ano = "red"


def enviar_emails():
    corpo_email = f"""
      <p>Bom dia,</p>
      <p>O resultado de ontem <strong>({dia_indicador.day}/{dia_indicador.month})</strong> da <strong>Loja {loja}</strong> foi:</p>

      <table>
      <tr>
        <th>Indicador</th>
        <th>Valor Dia</th>
        <th>Meta Dia</th>
        <th>Cenario Dia</th>
      </tr>
      <tr>
        <td>Faturamento</td>
        <td style="text-align: center">R${faturamento_dia:.2f}</td>
        <td style="text-align: center">R${meta_faturamento_dia}</td>
        <td style="text-align: center"><font color="{cor_fat_dia}">◙</font></td>
      </tr>
      <tr>
        <td>Diversidade de Produtos</td>
        <td style="text-align: center">{qtde_produtos_dia}</td>
        <td style="text-align: center">{meta_qtdeprodutos_dia}</td>
        <td style="text-align: center"><font color="{cor_qtde_dia}">◙</font></td>
      </tr>
      <tr>
        <td>Ticket Médio</td>
        <td style="text-align: center">R${ticket_medio_dia}</td>
        <td style="text-align: center">R${meta_ticketmedia_dia:.2f}</td>
        <td style="text-align: center"><font color="{cor_ticket_dia}">◙</font></td>
      </tr>
    </table>
    <br>
    <table>
      <tr>
        <th>Indicador</th>
        <th>Valor Ano</th>
        <th>Meta Ano</th>
        <th>Cenario Ano</th>
      </tr>
      <tr>
        <td>Faturamento</td>
        <td style="text-align: center">R${faturamento_ano}</td>
        <td style="text-align: center">R${meta_faturamento_ano}</td>
        <td style="text-align: center"><font color="{cor_fat_ano}">◙</font></td>
      </tr>
      <tr>
        <td>Diversidade de Produtos</td>
        <td style="text-align: center">{qtde_produtos_ano}</td>
        <td style="text-align: center">{meta_qtdeprodutos_ano}</td>
        <td style="text-align: center"><font color="{cor_qtde_ano}">◙</font></td>
      </tr>
      <tr>
        <td>Ticket Médio</td>
        <td style="text-align: center">R${ticket_medio_ano}</td>
        <td style="text-align: center">R${meta_ticketmedia_ano:.2f}</td>
        <td style="text-align: center"><font color="R${cor_ticket_ano}">◙</font></td>
      </tr>
    </table>

      <p>Segue em anexo a planilha com todos os dados para mais detalhes.</p>
      <p>Att., Guilherme</p>
      """

    msg = EmailMessage()
    msg['Subject'] = "OnePage Dia {}/{} - Loja{}".format(dia_indicador.day, dia_indicador.month, loja)
    msg['From'] = 'pandagamer207@gmail.com'
    msg['To'] = emails.loc[emails['Loja']==loja, 'E-mail'].values[0]
    password = 'gpzxgbnqnrsvjlwr'
    msg.add_header('Content-Type', 'text/html')
    msg.set_content(corpo_email, subtype='html')


    # ANEXO
    attachment = pathlib.Path.cwd() / caminho_backup / loja / f'{dia_indicador.month}_{dia_indicador.day}_{loja}.xlsx'
    
    mime_type, encoding = mimetypes.guess_type(attachment.name)
    with open(attachment, 'rb') as fp:
      dados = fp.read()
    msg.add_attachment(dados, filename=f'{dia_indicador.month}_{dia_indicador.day}_{loja}.xlsx', maintype='application', subtype='vnd.openxmlformats-officedocument.spreadsheetml.sheet')


    s = smtplib.SMTP('smtp.gmail.com: 587')
    s.starttls()
    
    # Login Credentials for sending the mail
    s.login(msg['From'], password)
    s.send_message(msg)
    print(f'Email  da {loja} enviado com sucesso!')


enviar_emails()

### Criando um ranking de lojas para a diretoria

In [ ]:
faturamento_lojas = vendas.groupby('Loja')[['Loja', 'Valor Final']].sum(numeric_only=True)
faturamento_lojas_ano = faturamento_lojas.sort_values(by='Valor Final', ascending=False)
display(faturamento_lojas)

nome_arquivo = '{}_{}_Rankking Anual.xlsx'.format(dia_indicador.month, dia_indicador.day)
faturamento_lojas_ano.to_excel(r'Backup Arquivos Lojas\{}'.format(nome_arquivo))

vendas_dia = vendas.loc[vendas['Data']==dia_indicador, :]
faturamento_lojas_dia = vendas_dia.groupby("Loja")[['Loja', 'Valor Final']].sum(numeric_only=True)
faturamento_lojas_dia = faturamento_lojas_dia.sort_values(by='Valor Final', ascending=False)
display(faturamento_lojas_dia)

nome_arquivo = '{}_{}_Rankking Dia.xlsx'.format(dia_indicador.month, dia_indicador.day)
faturamento_lojas_dia.to_excel(r'Backup Arquivos Lojas\{}'.format(nome_arquivo))


### Passo 6 - Enviando email para diretoria com os rankings


In [41]:
# ENVIAR EMAILS
def enviar_email():
    corpo_email = f"""
    <html>
    <body>
    <p>Prezados, bom dia</p>

    <p>
    Melhor loja do dia em Faturamento: Loja <strong>{faturamento_lojas_dia.index[0]}</strong> com Faturamento <strong>R${faturamento_lojas_dia.iloc[0, 0]:.2f}</strong><br>
    Pior loja do dia em Faturamento: Loja <strong>{faturamento_lojas_dia.index[-1]}</strong> com Faturamento <strong>R${faturamento_lojas_dia.iloc[-1, 0]:.2f}</strong>
    </p>

    <p>
    Melhor loja do ano em Faturamento: Loja <strong>{faturamento_lojas_ano.index[0]}</strong> com faturamento <strong>R${faturamento_lojas_ano.iloc[0, 0]:.2f}</strong><br>
    Pior loja do ano em Faturamento: Loja <strong>{faturamento_lojas_ano.index[-1]}</strong> com faturamento <strong>R${faturamento_lojas_ano.iloc[-1, 0]:.2f}</strong>
    </p>

    <p>Segue em anexo os rankings do ano e do dia de todas as lojas.</p>

    <p>Qualquer dúvida estou à disposição</p>

    <p>Att.,<br>Guilherme</p>
    </body>
    </html>
    """
    msg = EmailMessage()
    msg['Subject'] = "Ranking Dia {}/{} - Loja{}".format(dia_indicador.day, dia_indicador.month, loja)
    msg['From'] = 'pandagamer207@gmail.com'
    msg['To'] = emails.loc[emails['Loja']=='Diretoria', 'E-mail'].values[0]
    password = 'gpzxgbnqnrsvjlwr'
    msg.add_header('Content-Type', 'text/html')
    msg.set_content(corpo_email, subtype='html')
# ANEXO
    attachment = pathlib.Path.cwd() / caminho_backup / '{}_{}_Ranking Anual.xlsx'.format(dia_indicador.month, dia_indicador.day)
    mime_type, encoding = mimetypes.guess_type(attachment.name)
    with open(attachment, 'rb') as fp:
        dados = fp.read()
    msg.add_attachment(dados, filename= f'{dia_indicador.month}_{dia_indicador.day}_Ranking dia.xlsx', maintype='application', subtype='vnd.openxmlformats-officedocument.spreadsheetml.sheet')
    msg.add_attachment(dados, filename= f'{dia_indicador.month}_{dia_indicador.day}_Ranking Anual.xlsx', maintype='application', subtype='vnd.openxmlformats-officedocument.spreadsheetml.sheet')


    s = smtplib.SMTP('smtp.gmail.com: 587')
    s.starttls()

    # CREDENCIAIS DE LOGIN PARA ENVIO DE EMAIL
    s.login(msg['From'], password)
    s.send_message(msg)
    print(f'Email  da Diretoria enviado com sucesso!')
    
enviar_email()

Email  da Diretoria enviado com sucesso!
